In [5]:

import h5py,os
import numpy as np

mergecode='/capstor/store/cscs/userlab/s1174/lyan/code/projectData2/01_Nsgm/cB211.072.64_base/data_merge/Nsgm.h5'

basepath='/capstor/store/cscs/userlab/s1174/lyan/code/projectData2/01_Nsgm/cB211.072.64_base_'
tfss=['8,10,12,14','16,18,20']
# tfss=['8,10,12,14,16,18,20']
# tfss=['8','10','12','14']

NJNpath='/capstor/store/cscs/userlab/s1174/lyan/code/projectData2/01_Nsgm/cB211.072.64_base/data_from_others/thrp-conn.h5'

cfg2old=lambda cfg: cfg[1:]+'_r'+{'a':'0','b':'1','c':'2','d':'3'}[cfg[0]]
cfg2new=lambda cfg: {'0':'a','1':'b','2':'c','3':'d'}[cfg[-1]] + cfg[:4]
with open('data_aux/cfgs_run','r') as f:
    cfgs=f.read().splitlines()
cfgs_old=[cfg2old(cfg) for cfg in cfgs]

flag=True
with h5py.File(mergecode,'w') as fw, h5py.File(NJNpath) as fnjn:    
    
    for tfs in tfss[:1]:
        files=[file for file in os.listdir(basepath+tfs+'/data_merge/') if '.h5' in file]
        assert(len(files)==1)
        with h5py.File(basepath+tfs+'/data_merge/'+files[0]) as fr:
            cfgs0=[cfg.decode() for cfg in fr['cfgs']]
            assert(np.all(cfgs==cfgs0))
            for key in fr.keys():       
                if key=='diags':
                    continue             
                fw.copy(fr[key],fw,name=key)
            for diag in fr['diags'].keys():
                if diag.split('-')[0] in ['NJN','B3pt','W3pt','Z3pt'] or\
                    'j' in diag.split('-') or 'jPi' in diag.split('-') or 'jPf' in diag.split('-'):
                    for ky in fr['diags'][diag].keys():
                        if ky in ['data']:
                            continue
                        if diag=='NJN' and ky=='srcs':
                            continue
                        fw.copy(fr['diags'][diag][ky],fw,name=f'diags/{diag}/{ky}')
                    continue
                fw.copy(fr['diags'][diag],fw,name=f'diags/{diag}')
            
            
    for tfs in tfss[:]:
        files=[file for file in os.listdir(basepath+tfs+'/data_merge/') if '.h5' in file]
        assert(len(files)==1)
        with h5py.File(basepath+tfs+'/data_merge/'+files[0]) as fr:
            cfgs0=[cfg.decode() for cfg in fr['cfgs']]
            assert(np.all(cfgs==cfgs0))
            for diag in fr['diags'].keys():
                # if diag == 'NJN':
                #     # print(fr['diags'][diag]['opabs'][:])
                #     for fla in fr['diags'][diag]['data'].keys():
                #         tf=int(fla.split('_')[-1])
                #         t=np.array([fnjn[f'gS/up/dt{tf}/{cfg}'][:]+fnjn[f'gS/dn/dt{tf}/{cfg}']  for cfg in cfgs_old])
                #         t=t[:,:,None,None]
                #         t=t[:,:,[0]*4]
                #         t=t[:,:,:,[0]*16]
                #         fw.create_dataset(f'diags/{diag}/data/{fla}',data=t)
                #     continue
                
                if diag.split('-')[0] in ['NJN','B3pt','W3pt','Z3pt'] or\
                    'j' in diag.split('-') or 'jPi' in diag.split('-') or 'jPf' in diag.split('-'):
                    for fla in fr['diags'][diag]['data'].keys():
                        try:
                            fw.create_dataset(f'diags/{diag}/data/{fla}',data=fr[f'diags/{diag}/data/{fla}'])
                        except:
                            if 'js' in fla or 'jc' in fla:
                                pass
                            else:
                                print(f'diags/{diag}/data/{fla}')
                    # for file in fr['diags'][diag]['srcs'].keys():
                    #     if file not in fw['diags'][diag]['srcs'].keys():
                    #         fw.create_dataset(f'diags/{diag}/scrs/{file}',data=fr['diags'][diag]['srcs'][file])


In [13]:
path='/capstor/store/cscs/userlab/s1174/lyan/code/projectData2/01_Nsgm/cB211.072.64_base/data_merge/Nsgm.h5'
NJNpath='/capstor/store/cscs/userlab/s1174/lyan/code/projectData2/01_Nsgm/cB211.072.64_base/data_from_others/thrp-conn.h5'
with h5py.File(path) as f, h5py.File(NJNpath) as fn:
    print(f['diags/NJN/data/p_j+_p_deltat_10'][0,:,0,0]*2)

[7.29153893e-09 8.58386695e-09 8.76655887e-09 7.88862325e-09
 7.05533387e-09 6.39252312e-09 6.15194434e-09 6.27519062e-09
 6.25901028e-09 6.59050101e-09 5.86377596e-09]


In [ ]:
path='/capstor/store/cscs/userlab/s1174/lyan/code/projectData2/01_Nsgm/cB211.072.64_base/data_from_others/thrp-conn.h5'
with h5py.File(path) as f:
    print(f['gS/up/dt8/0004_r1'])
    t=f['gS/up']
    

<HDF5 dataset "0004_r1": shape (9,), type "<f8">
